<a href="https://colab.research.google.com/github/JHyunjun/SNU/blob/main/gtsrb_cnn_98_test_accuracy_hj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Traffic Sign Recognizer - 99% accuracy*

## Importing Required Libraries

In [113]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
np.random.seed(7)

from matplotlib import style
style.use('fivethirtyeight')

In [114]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path_to_zip_file = '/content/drive/MyDrive/Colab Notebooks/snu/project/data.zip'
directory_to_extract_to = '/content/drive/MyDrive/Colab Notebooks/snu/project/data'

import zipfile
with zipfile.ZipFile(path_to_zip_file,'r') as zip_ref:
  zip_ref.extractall(directory_to_extract_to)


## Assigning Path for Dataset

In [ ]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/snu/project/data/data'
data_dir_test = '/content/drive/MyDrive/Colab Notebooks/snu/project/datafortest'

train_path = '/content/drive/MyDrive/Colab Notebooks/snu/project/data/data/Train'
test_path = '/content/drive/MyDrive/Colab Notebooks/snu/project/datafortest/Test'

# Resizing the images to 40x40x3
IMG_HEIGHT = 40
IMG_WIDTH = 40
channels = 3

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = 'hyunjunjang'
os.environ['KAGGLE_KEY'] = '63c6ccdb188e6b6557e021a25bf8f37d'

In [ ]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

In [ ]:
path_to_zip_file = '/content/gtsrb-german-traffic-sign.zip'
directory_to_extract_to = '/content/drive/MyDrive/Colab Notebooks/snu/project/datafortest'

import zipfile
with zipfile.ZipFile(path_to_zip_file,'r') as zip_ref:
  zip_ref.extractall(directory_to_extract_to)

## Finding Total Classes

In [ ]:
NUM_CATEGORIES = len(os.listdir(train_path))
NUM_CATEGORIES

In [ ]:
# Label Overview
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

## Visualizing The Dataset

In [ ]:
folders = os.listdir(train_path)

train_number = []
class_num = []

for folder in folders:
    train_files = os.listdir(train_path + '/' + folder)
    train_number.append(len(train_files))
    class_num.append(classes[int(folder)])
    
# Sorting the dataset on the basis of number of images in each class
zipped_lists = zip(train_number, class_num)
sorted_pairs = sorted(zipped_lists)
tuples = zip(*sorted_pairs)
train_number, class_num = [ list(tuple) for tuple in  tuples]

# Plotting the number of images in each class
plt.figure(figsize=(21,10))  
plt.bar(class_num, train_number)
plt.xticks(class_num, rotation='vertical')
plt.show()

In [ ]:
# Visualizing 25 random images from test data
import random
from matplotlib.image import imread

test = pd.read_csv(data_dir_test + '/Test.csv')
imgs = test["Path"].values

plt.figure(figsize=(25,25))

for i in range(1,26):
    plt.subplot(5,5,i)
    random_img_path = data_dir_test + '/' + random.choice(imgs)
    rand_img = imread(random_img_path)
    plt.imshow(rand_img)
    plt.grid(b=None)
    plt.xlabel(rand_img.shape[1], fontsize = 20)#width of image
    plt.ylabel(rand_img.shape[0], fontsize = 20)#height of image


## Collecting the Training Data

In [ ]:
image_data = []
image_labels = []

for i in range(NUM_CATEGORIES):
    path = data_dir + '/Train/' + str(i)
    images = os.listdir(path)

    for img in images:
        try:
            image = cv2.imread(path + '/' + img)
            image_fromarray = Image.fromarray(image, 'RGB')
            resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
            image_data.append(np.array(resize_image))
            image_labels.append(i)
        except:
            print("Error in " + img)

# Changing the list to numpy array
image_data = np.array(image_data)
image_labels = np.array(image_labels)

print(image_data.shape, image_labels.shape)

## Shuffling the training data

In [ ]:
shuffle_indexes = np.arange(image_data.shape[0])
np.random.shuffle(shuffle_indexes)
image_data = image_data[shuffle_indexes]
image_labels = image_labels[shuffle_indexes]

## Splitting the data into train and validation set

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(image_data, image_labels, test_size=0.1, random_state=42, shuffle=True)

X_train = X_train/255 
X_val = X_val/255

print("X_train.shape", X_train.shape)
print("X_valid.shape", X_val.shape)
print("y_train.shape", y_train.shape)
print("y_valid.shape", y_val.shape)

## One hot encoding the labels

In [ ]:
y_train = keras.utils.to_categorical(y_train, NUM_CATEGORIES)
y_val = keras.utils.to_categorical(y_val, NUM_CATEGORIES)

print(y_train.shape)
print(y_val.shape)

## Making the model

In [ ]:
model = keras.models.Sequential([    
    keras.layers.Conv2D(filters=16*2, kernel_size=(3,3), activation='relu', input_shape=(IMG_HEIGHT,IMG_WIDTH,channels)),
    keras.layers.Conv2D(filters=32*2, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),
    
    keras.layers.Conv2D(filters=64*2, kernel_size=(3,3), activation='relu'),
    keras.layers.Conv2D(filters=128*2, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    keras.layers.BatchNormalization(axis=-1),
    
    keras.layers.Flatten(),
    keras.layers.Dense(512*2, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=0.5),
    
    keras.layers.Dense(43, activation='softmax')
])

In [ ]:
lr = 0.001
epochs = 10

opt = Adam(lr=lr, decay=lr / (epochs * 0.5))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
# JY's code ; SMOTE for oversampling

from imblearn.over_sampling import SMOTE

data = X_train
labels = y_train

def smote(data, labels): 
  
  X = data.reshape(len(data), -1)
  y = labels
  smt = SMOTE(sampling_strategy='not majority', k_neighbors=5, n_jobs=-1)
  X_smote, y_smote = smt.fit_resample(X, y)
  X_smote = X_smote.reshape((-1, )+ data.shape[1:]) 

  return  X_smote, y_smote

data, labels = smote(data, labels)
history = model.fit(data, labels, batch_size = 32, epochs = epochs, validation_data = (X_val, y_val))

'''
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

# fit the model
history = model.fit(data, labels, validation_data = (X_val, y_val), epochs=1, verbose=0)

# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_val, y_val, verbose=0)
'''

## Augmenting the data and training the model

In [ ]:
aug = ImageDataGenerator(
    rotation_range = 10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

history = model.fit(aug.flow(X_train, y_train, batch_size = 32), epochs=epochs, validation_data=(X_val, y_val))

## Evaluating the model

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
# Evaluation HJ
test = pd.read_csv(data_dir_test + '/Test.csv')
imgs = test["Path"].values
data =[]

for img in imgs:
    try:
        image = cv2.imread(data_dir_test + '/' +img)
        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
        data.append(np.array(resize_image))
    except:
        print("Error in " + img)

X_test = np.array(data)
X_test = X_test/255

pred = model.predict(X_test)
classes_x = np.argmax(pred,axis=1)

## Loading the test data and running the predictions

In [ ]:
test = pd.read_csv(data_dir_test + '/Test.csv')

labels = test["ClassId"].values
imgs = test["Path"].values

data =[]

for img in imgs:
    try:
        image = cv2.imread(data_dir_test + '/' +img)
        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
        data.append(np.array(resize_image))
    except:
        print("Error in " + img)
X_test = np.array(data)
X_test = X_test/255

pred = model.predict(X_test)
pred = np.argmax(pred, axis=1)

#Accuracy with the test data
print('Test Data accuracy: ',accuracy_score(labels, pred)*100)

## Visualizing the confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(labels, pred)

In [ ]:
import seaborn as sns
df_cm = pd.DataFrame(cf, index = classes,  columns = classes)
plt.figure(figsize = (20,20))
sns.heatmap(df_cm, annot=True)

## Classification report

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(labels, pred))

## Predictions on Test Data

In [ ]:
plt.figure(figsize = (25, 25))

start_index = 0
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    prediction = pred[start_index + i]
    actual = labels[start_index + i]
    col = 'g'
    if prediction != actual:
        col = 'r'
    plt.xlabel('Actual={} || Pred={}'.format(actual, prediction), color = col)
    plt.imshow(X_test[start_index + i])
plt.show()

In [ ]:
# For Suggestion

suggest_test = '/content/drive/MyDrive/Colab Notebooks/snu/project/data/data'

# Resizing the images to 40x40x3
IMG_HEIGHT = 40
IMG_WIDTH = 40
channels = 3

image_data = []
image_predict = []


path = suggest_test + '/Test/'
images = os.listdir(path)

for img in images:
    try:
        image = cv2.imread(path + '/' + img)
        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
        image_data.append(np.array(resize_image))
        image_predict.append(img)
    except:
        print(i," iteration"," and Error in " + img)

X_test = np.array(image_data)
X_test = X_test/255

pred_suggest = model.predict(X_test)
suggest = np.argmax(pred_suggest,axis=1)

In [ ]:
image_predict = np.array(image_predict)

print(image_predict.shape)
print(suggest.shape)
print(X_test.shape)

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/snu/project/data/data

In [ ]:
# Save data as csv

import pandas as pd

data = suggest
df = pd.DataFrame(data)
df2 = pd.DataFrame(image_predict)
df.to_csv('HD_proj_team1_1.csv', index = False)
df2.to_csv('HD_proj_team1_1_image_name.csv', index = False)